# Load Model

In [ ]:
original_model_name = "CartPole_ppo"


In [ ]:
if True: # if to generate a model with our own architecture

    from automl.ml.models.neural_model import FullyConnectedModelSchema
    from automl.rl.environment.gymnasium_env import GymnasiumEnvironmentWrapper

    
    environment = GymnasiumEnvironmentWrapper({"environment" : "CartPole-v1", "device" : "cpu"})

    original_model = FullyConnectedModelSchema(
        {
            "hidden_layers" : 2,
            "hidden_size" : 64,
            "device" : "cpu",

            "output_shape" : environment.get_agent_action_space("agent_1"),
            "input_shape" : environment.get_agent_state_space("agent_1")
        }
    )

    original_model_name = "FC_CartPole_ppo"

In [ ]:
if False: # if to load model from SB3

    from automl.external_support.sb3.sb3_model_wrapper import SB3WrapperTorch

    #original_model = SB3WrapperTorch({"sb3_model" : "dqn-MountainCar-v0", "device" : "cpu"})

    #original_model = SB3WrapperTorch({"sb3_model" : "dqn-CartPole-v1", "device" : "cpu"})
    #original_model_name = "sb3_CartPoler_dqn"

    original_model = SB3WrapperTorch({"sb3_model" : "ppo-CartPole-v1", "device" : "cpu"})
    original_model_name = "sb3_CartPole_ppo"


In [ ]:
#original_model_name = "sb3_MountainCar_dqn"
model_name = original_model_name

In [ ]:
from automl.ml.models.torch_model_components import TorchModelComponent

loaded_model : TorchModelComponent = original_model.clone()

# Perturb Models

In [ ]:
execute_forget = False
execute_perturb = False
execute_gaussian = True

## Partially forget parameters (reinitialize them)

In [ ]:
percentage_to_forget = 100
std_of_reinit = 0.00000000000000001

In [ ]:
if execute_forget == True:

    from automl.ml.models.torch_model_utils import perturb_model_parameters_partial_forgetting
    perturb_model_parameters_partial_forgetting(loaded_model, fraction=percentage_to_forget / 100.0, std=std_of_reinit)

    loaded_model.write_line_to_notes(f"Made model forget {percentage_to_forget}% of its parameters with a std or reinitialization of {std_of_reinit}", True)

    model_name = f"{model_name}_forgotten_{percentage_to_forget}_{std_of_reinit}"

## Perturb models randomly by percentage

In [ ]:
max_percentage = 50
min_percentage = 10

In [ ]:
if execute_perturb:

    from automl.ml.models.torch_model_utils import perturb_model_parameters
    
    perturb_model_parameters(loaded_model, min_percentage / 100.0, max_percentage / 100.0)
    
    loaded_model.write_line_to_notes(f"Perturbed model parameters by a percentage [{min_percentage}%, {max_percentage}%], in either direction", use_datetime=True)
    
    model_name = f"{model_name}_perturbed_{min_percentage}_{max_percentage}"

## Perturb model with gaussian noise

In [ ]:
avg = 0

std = 0.8
#std = 0.05

#fraction = 0.8
fraction = 0.9

In [ ]:
if execute_gaussian:

    from automl.ml.models.torch_model_utils import perturb_model_parameters_gaussian
    
    perturb_model_parameters_gaussian(loaded_model, avg, std, fraction)
    
    loaded_model.write_line_to_notes(f"Perturbed model parameters by adding gaussian noise with average {avg} and standard deviation of {std} to a fraction {fraction} of its values", use_datetime=True)
    
    model_name = f"{model_name}_gaussian_{avg}_{std}_{fraction}"


# Study and save model

## Note the difference between the models

In [ ]:
from automl.ml.models.torch_model_utils import model_parameter_distance

l2_distance, avg_distance, cosine_sim = model_parameter_distance(original_model, loaded_model)

print(f"L2 distance between original and perturbed model: {l2_distance}")
print(f"Average distance between original and perturbed model: {avg_distance}")
print(f"Cosine similarity between original and perturbed model: {cosine_sim}")

loaded_model.write_line_to_notes(f"After changes to the model, noting the differences between it and the original model\n    L2 distance: {l2_distance}\n    Avg L2 distance (divided by total parameters): {avg_distance}\n    Cosine similarity: {cosine_sim}", use_datetime=True)

## Total Changes done to model

In [ ]:
print("Model notes:\n" + loaded_model.get_notes_as_text())

## Save Model

In [ ]:
models_path = "data\\models"
models_path = f"{models_path}\\{original_model_name}"



loaded_model.pass_input({"base_directory" : models_path,
                         "artifact_relative_directory": model_name,
                           "name" : original_model_name})

model_store_path = loaded_model.get_artifact_directory()

In [ ]:
loaded_model.save_state(True)